
# Game-playing Agent

Notebook version solving the project '**Build a Game-playing Agent**' from Udacity's Artificial Intelligence Nanodegree <br>
Source: https://github.com/udacity/AIND-Isolation

**Goal**: Implement an effective adversarial search agent to play the game **Isolation**

Isolation is a deterministic, two-player game of perfect information in which the players alternate turns moving a single piece from one cell to another on a board.  Whenever either player occupies a cell, that cell becomes blocked for the remainder of the game.  The first player with no remaining legal moves loses, and the opponent is declared the winner.

This project uses a version of Isolation where each agent is restricted to L-shaped movements (like a knight in chess) on a rectangular grid (like a chess or checkerboard).  The agents can move to any open cell on the board that is 2-rows and 1-column or 2-columns and 1-row away from their current position on the board. Movements are blocked at the edges of the board (the board does not wrap around), however, the player can "jump" blocked or occupied spaces (just like a knight in chess).

Additionally, agents will have a fixed time limit each turn to search for the best move and respond.  If the time limit expires during a player's turn, that player forfeits the match, and the opponent wins.

These rules are implemented in the `isolation.Board` class provided in the repository. 

The final evaluation will estimate the strength rating of student-agent with iterative deepening and
a custom heuristic evaluation function against fixed-depth minimax and
alpha-beta search agents by running a round-robin tournament for the student
agent. The student agent plays a fixed number of "fair" matches against each test
agent. The matches are fair because the board is initialized randomly for both
players, and the players play each match twice -- switching the player order
between games. This helps to correct for imbalances in the game due to both
starting position and initiative.

For example, if the random moves chosen for initialization are (5, 2) and
(1, 3), then the first match will place agentA at (5, 2) as player 1 and
agentB at (1, 3) as player 2 then play to conclusion; the agents swap
initiative in the second match with agentB at (5, 2) as player 1 and agentA at
(1, 3) as player 2.

## Sample players

Collection of player classes for comparison with the custom agent and example heuristic functions

In [1]:
from random import randint

def null_score(game, player):
    """This heuristic presumes no knowledge for non-terminal states, and
    returns the same uninformative value for all other states. """

    if game.is_loser(player):
        return float("-inf")
    if game.is_winner(player):
        return float("inf")
    return 0.


def open_move_score(game, player):
    """The basic evaluation function described in lecture that outputs a score
    equal to the number of moves open for your computer player on the board. """

    if game.is_loser(player):
        return float("-inf")
    if game.is_winner(player):
        return float("inf")
    return float(len(game.get_legal_moves(player)))


def improved_score(game, player):
    """The "Improved" evaluation function discussed in lecture that outputs a
    score equal to the difference in the number of moves available to the
    two players.   """
    
    if game.is_loser(player):
        return float("-inf")
    if game.is_winner(player):
        return float("inf")
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - opp_moves)


class RandomPlayer():
    """Player that chooses a move randomly."""

    def get_move(self, game, legal_moves, time_left):
        """Randomly select a move from the available legal moves."""

        if not legal_moves:
            return (-1, -1)
        return legal_moves[randint(0, len(legal_moves) - 1)]


class GreedyPlayer():
    """Player that chooses next move to maximize heuristic score. This is
    equivalent to a minimax search agent with a search depth of one.
    """

    def __init__(self, score_fn=open_move_score):
        self.score = score_fn

        
    def get_move(self, game, legal_moves, time_left):

        if not legal_moves:
            return (-1, -1)
        _, move = max([(self.score(game.forecast_move(m), self), m) for m in legal_moves])
        return move


class HumanPlayer():
    """Player that chooses a move according to user's input."""

    def get_move(self, game, legal_moves, time_left):
        """
        Select a move from the available legal moves based on user input at the
        terminal.  If testing with this player, remember to disable move timeout in
        the call to `Board.play()`. """

        if not legal_moves:
            return (-1, -1)

        print(('\t'.join(['[%d] %s' % (i, str(move)) for i, move in enumerate(legal_moves)])))

        valid_choice = False
        while not valid_choice:
            try:
                index = int(input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')

            except ValueError:
                print('Invalid index! Try again.')

        return legal_moves[index]

## Custom heuristic score: Lock, mobility, and moves with tuned linear function


Zero sum function based on player’s mobility (moves / blank_spaces ), and also predicting if one player can lock the other in its next ply:

- Predict if the player is about to win or lose in the next play when any player has no move available for its next play (cases not detected by game.is_winner() or game.is_loser() due their inactive/active player condition): +inf, -inf

- Predict, when a player only has one move available, if the other can reach  that cell before by looking among its available moves, thus finishing the   game: +inf, inf

- The usual scenario is valued as: `score = A * own_mobility + B * opp_mobility + C * own_moves + D * opp_moves`, being the mobility defined as the ratio between the available move for the player and the total available cells of the board, thus the value for looking for more legal moves will be increased when the game is about to end for lack of empty cells. After a few tournaments, the selected parameters were: `A = 10, B = -10, C = 1, D = 0`



In [2]:
def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """

    if game.is_loser(player):
        return float("-inf")
    if game.is_winner(player):
        return float("inf")

    own_moves_list = game.get_legal_moves(player)
    own_moves = len(own_moves_list)

    opp_moves_list = (game.get_legal_moves(game.get_opponent(player)))
    opp_moves = len(opp_moves_list)

    # game.is_loser() game.is_winner depends on the active player.
    # These cases can occur. e.g.: Opponent has no moves while Own is active
    if opp_moves == 0:
        return float("inf")
    if own_moves == 0:
        return float("-inf")
    # Thus we can predict 1 ply more

    # This easy killing move  can also be predicted:
    # Lock the opponent if, in your turn, you can get to its unique move
    if (player == game.active_player and opp_moves == 1 and
        opp_moves_list[0] in own_moves_list):
        return float("inf")
    # opposite case
    if (player != game.active_player and own_moves == 1 and
        own_moves_list[0] in opp_moves_list):
        return float("-inf")

    # heuristic values for the rest of the cases
    blank_spaces = game.width * game.height - game.move_count
    own_mobility = own_moves / blank_spaces
    opp_mobility = opp_moves / blank_spaces

    score = 10*(own_mobility - opp_mobility) + own_moves

    return float(score)

## Custom Player

Game-playing agent that chooses a move using your evaluation function and a depth-limited minimax algorithm with alpha-beta pruning; also making sure it properly uses minimax and alpha-beta to return a good move before the search time limit expires.
    

- `CustomPlayer.minimax()`:   implement minimax search
- `CustomPlayer.alphabeta()`: implement minimax search with alpha-beta pruning
- `CustomPlayer.get_move()`:  implement fixed-depth and iterative deepening search

In [3]:
class CustomPlayer:
    """
    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    iterative : boolean (optional)
        Flag indicating whether to perform fixed-depth search (False) or
        iterative deepening search (True).

    method : {'minimax', 'alphabeta'} (optional)
        The name of the search method to use in get_move().

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """

    def __init__(self, search_depth=3, score_fn=custom_score,
                 iterative=True, method='minimax', timeout=10.):
        self.search_depth = search_depth
        self.iterative = iterative
        self.score = score_fn
        self.method = method
        self.time_left = None
        self.TIMER_THRESHOLD = timeout


    def get_move(self, game, legal_moves, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """

        self.time_left = time_left

        move = None

        if not legal_moves:
            return -1, -1

        try:

            if self.iterative:  # Iterative search
                depth = 1

                # Get the best move found (higher value) evaluating each level
                # (from top to bottom) before searching in the next level.
                # Timeout or an optimal state found (e.g. win) returns
                while True:
                    if self.method == 'minimax':
                        score, move = self.minimax(game, depth)
                    elif self.method == 'alphabeta':
                        score, move = self.alphabeta(game, depth)
                    depth += 1
                    # No need to wait for Timeout if an optimal move is reached
                    if score == float('inf'):
                        return move
            else:   # Depth-first search
                if self.method == 'minimax':
                    _, move = self.minimax(game, self.search_depth)
                elif self.method == 'alphabeta':
                    _, move = self.alphabeta(game, self.search_depth)

        except Timeout:
            pass

        # Return the best move from the last completed search iteration
        return move


    def minimax(self, game, depth, maximizing_player=True):
        """Implement the minimax search algorithm as described in the lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        -------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves

        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()

        legal_moves = game.get_legal_moves()
        move = (-1, -1)

        # If no more moves -> Terminal state: return its score with move (-1,-1)
        if not legal_moves:
            return self.score(game, self), move

        # depth 0 evaluates and returns directly the value for the current state
        # The current location here is the move to return
        #  (it won't be evaualted again since depth = 0)
        if depth == 0:
            score = self.score(game, self)
            move = game.get_player_location(self)
            return score, move

        depth -= 1  # Each recursive call to minimax will have one less depth

        if maximizing_player:
            # MAX-VALUE node: Get the maximum value from the MIN-VALUE nodes of
            #  its next level
            score, move = max([(self.minimax(game.forecast_move(m),
                depth, False)[0], m) for m in legal_moves])
        else:
            # MIN-VALUE node: Get the minimum value from the MAX-VALUE nodes of
            #  its next level
            score, move = min([(self.minimax(game.forecast_move(m),
                depth, True)[0], m) for m in legal_moves])
        return score, move


    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf"),
                  maximizing_player=True):
        """Implement minimax search with alpha-beta pruning as described in the
        lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        -------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()


        legal_moves = game.get_legal_moves()
        move = (-1, -1)
        score = None

        # If no more moves -> Terminal state: return its score with move (-1,-1)
        if not legal_moves:
            return self.score(game, self), move

        # depth 0 evaluates and returns directly the value for the current state
        # The current location here is the move to return
        #  (it won't be evaluated  again since depth = 0)
        if depth == 0:
            score = self.score(game, self)
            move = game.get_player_location(self)  # Todo
            return score, move

        depth -= 1  # Each recursive call to alphabeta will have one less depth

        # A dict with key=move and value=score is used for pruning the branches
        scores = {}
        if maximizing_player:  # MAX-VALUE node
            for m in legal_moves:
                # evaluate the next level of m (minimizing player)
                #  and return its scores dictionary
                scores[m], _ = self.alphabeta(game.forecast_move(m), depth,
                                              alpha, beta, False)
                # get the max value of the resulted scores dictionary for the
                #  current level(maximizing player)
                move = max(scores, key=scores.get)
                score = scores[move]

                # Prune the next branches (for the next m's) if the value
                #  obtained so far is larger than beta
                #  (best choice for MIN in the higher level).
                if score >= beta:
                    return score, move

                # Update alpha (best choice for MAX) for the next evaluation
                #  (minimizing player)
                alpha = max(alpha, score)

        else:
            for m in legal_moves:  # MIN-VALUE node
                # evaluate the next level of m (maximizing player) and return
                #  its scores dictionary
                scores[m], _ = self.alphabeta(game.forecast_move(m), depth,
                                              alpha, beta, True)
                # get the min value of the resulted scores dictionary for the
                #  current level(minimizing player)
                move = min(scores, key=scores.get)
                score = scores[move]

                # Prune the next branches (for the next m's) if the value
                #  obtained so far is smaller than alpha
                #  (best choice for MAX in the higher level).
                if score <= alpha:
                    return score, move

                # Update beta (best choice for MIN) for the next evaluation
                #  (maximazing player)
                beta = min(beta, score)

        return score, move

## Game setup

In [4]:
import itertools
import random
import warnings

from collections import namedtuple

from isolation import Board

NUM_MATCHES = 5  # number of matches against each opponent
TIME_LIMIT = 30  # number of milliseconds before timeout

TIMEOUT_WARNING = "One or more agents lost a match this round due to " + \
                  "timeout. The get_move() function must return before " + \
                  "time_left() reaches 0 ms. You will need to leave some " + \
                  "time for the function to return, and may need to " + \
                  "increase this margin to avoid timeouts during  " + \
                  "tournament play."


Agent = namedtuple("Agent", ["player", "name"])

class Timeout(Exception):
    """Subclass base exception for code clarity."""
    pass

Play a "fair" set of matches between two agents by playing two games between the players, forcing each agent to play from randomly selected positions. This should control for differences in outcome resulting from advantage due to starting position on the board.

In [5]:
def play_match(player1, player2):

    num_wins = {player1: 0, player2: 0}
    num_timeouts = {player1: 0, player2: 0}
    num_invalid_moves = {player1: 0, player2: 0}
    games = [Board(player1, player2), Board(player2, player1)]

    # initialize both games with a random move and response
    for _ in range(2):
        move = random.choice(games[0].get_legal_moves())
        games[0].apply_move(move)
        games[1].apply_move(move)

    # play both games and tally the results
    for game in games:
        winner, _, termination = game.play(time_limit=TIME_LIMIT)

        if player1 == winner:
            num_wins[player1] += 1

            if termination == "timeout":
                num_timeouts[player2] += 1
            else:
                num_invalid_moves[player2] += 1

        elif player2 == winner:

            num_wins[player2] += 1

            if termination == "timeout":
                num_timeouts[player1] += 1
            else:
                num_invalid_moves[player1] += 1

    if sum(num_timeouts.values()) != 0:
        warnings.warn(TIMEOUT_WARNING)

    return num_wins[player1], num_wins[player2]

 Play one round (i.e., a single match between each pair of opponents)

In [6]:
def play_round(agents, num_matches):

    agent_1 = agents[-1]
    wins = 0.
    total = 0.

    print("\nPlaying Matches:")
    print("----------")

    for idx, agent_2 in enumerate(agents[:-1]):

        counts = {agent_1.player: 0., agent_2.player: 0.}
        names = [agent_1.name, agent_2.name]
        print("  Match {}: {!s:^11} vs {!s:^11}".format(idx + 1, *names), end=' ')

        # Each player takes a turn going first
        for p1, p2 in itertools.permutations((agent_1.player, agent_2.player)):
            for _ in range(num_matches):
                score_1, score_2 = play_match(p1, p2)
                counts[p1] += score_1
                counts[p2] += score_2
                total += score_1 + score_2

        wins += counts[agent_1.player]

        print("\tResult: {} to {}".format(int(counts[agent_1.player]),
                                          int(counts[agent_2.player])))

    return 100. * wins / total

## Evaluation


This section evaluates the performance of the custom heuristic function by
comparing the strength of an agent using iterative deepening (ID) search with
alpha-beta pruning against the strength rating of agents using other heuristic
functions.  The `ID_Improved` agent provides a baseline by measuring the
performance of a basic agent using Iterative Deepening and the "improved"
heuristic (from lecture) on your hardware.  The `Student` agent then measures
the performance of Iterative Deepening and the custom heuristic against the
same opponents.


The performance of time-limited iterative deepening search is hardware dependent (faster hardware is expected to search deeper than slower hardware in the same amount of time).  The script controls for these effects by also measuring the baseline performance of an agent called "ID_Improved" that uess Iterative Deepening and the improved_score heuristic from the above sample players.  

The tournament opponents are listed below: 

- Random: An agent that randomly chooses a move each turn.
- MM_Null: CustomPlayer agent using fixed-depth minimax search and the null_score heuristic
- MM_Open: CustomPlayer agent using fixed-depth minimax search and the open_move_score heuristic
- MM_Improved: CustomPlayer agent using fixed-depth minimax search and the improved_score heuristic
- AB_Null: CustomPlayer agent using fixed-depth alpha-beta search and the null_score heuristic
- AB_Open: CustomPlayer agent using fixed-depth alpha-beta search and the open_move_score heuristic
- AB_Improved: CustomPlayer agent using fixed-depth alpha-beta search and the improved_score heuristic



In [7]:
HEURISTICS = [("Null", null_score),
                  ("Open", open_move_score),
                  ("Improved", improved_score)]
AB_ARGS = {"search_depth": 5, "method": 'alphabeta', "iterative": False}
MM_ARGS = {"search_depth": 3, "method": 'minimax', "iterative": False}
CUSTOM_ARGS = {"method": 'alphabeta', 'iterative': True}

# Create a collection of CPU agents 
mm_agents = [Agent(CustomPlayer(score_fn=h, **MM_ARGS),
                   "MM_" + name) for name, h in HEURISTICS]
ab_agents = [Agent(CustomPlayer(score_fn=h, **AB_ARGS),
                   "AB_" + name) for name, h in HEURISTICS]
random_agents = [Agent(RandomPlayer(), "Random")]

# ID_Improved agent is used for comparison to the performance of the
# submitted agent for calibration on the performance across different
# systems; i.e., the performance of the student agent is considered
# relative to the performance of the ID_Improved agent to account for
# faster or slower computers.
test_agents = [Agent(CustomPlayer(score_fn=improved_score, **CUSTOM_ARGS), "ID_Improved"),
               Agent(CustomPlayer(score_fn=custom_score, **CUSTOM_ARGS), "Student")]

for agentUT in test_agents:
    print("")
    print("*************************")
    print("{:^25}".format("Evaluating: " + agentUT.name))
    print("*************************")

    agents = random_agents + mm_agents + ab_agents + [agentUT]
    win_ratio = play_round(agents, NUM_MATCHES)

    print("\n\nResults:")
    print("----------")
    print("{!s:<15}{:>10.2f}%".format(agentUT.name, win_ratio))



*************************
 Evaluating: ID_Improved 
*************************

Playing Matches:
----------
  Match 1: ID_Improved vs   Random    	Result: 19 to 1
  Match 2: ID_Improved vs   MM_Null   	Result: 19 to 1
  Match 3: ID_Improved vs   MM_Open   	Result: 12 to 8
  Match 4: ID_Improved vs MM_Improved 	Result: 12 to 8
  Match 5: ID_Improved vs   AB_Null   	Result: 14 to 6
  Match 6: ID_Improved vs   AB_Open   	Result: 19 to 1
  Match 7: ID_Improved vs AB_Improved 	Result: 20 to 0


Results:
----------
ID_Improved         82.14%

*************************
   Evaluating: Student   
*************************

Playing Matches:
----------
  Match 1:   Student   vs   Random    	Result: 20 to 0
  Match 2:   Student   vs   MM_Null   	Result: 19 to 1
  Match 3:   Student   vs   MM_Open   	Result: 18 to 2
  Match 4:   Student   vs MM_Improved 	Result: 15 to 5
  Match 5:   Student   vs   AB_Null   	Result: 15 to 5
  Match 6:   Student   vs   AB_Open   	Result: 20 to 0
  Match 7:   Student